### Week 4 Application Assignment

1. Build a full classification tree model with Reward column as the target variable and the following set of predictor variables: Salerank, X2013USSales, X2013WorldSales, ProfitMargin, NumStores, and IndustryType. Which feature has the highest feature importance? 

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from xgboost import to_graphviz, plot_importance

%matplotlib inline
sns.set_style('dark')
sns.set(font_scale=1.2)

from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error, mean_squared_error,r2_score
from sklearn.metrics import plot_confusion_matrix, plot_precision_recall_curve, plot_roc_curve, accuracy_score
from sklearn.metrics import auc, f1_score, precision_score, recall_score, roc_auc_score

#import feature_engine.missing_data_imputers as mdi
#from feature_engine.outlier_removers import Winsorizer

#from tpot import TPOTClassifier, TPOTRegressor
#from imblearn.under_sampling import RandomUnderSampler
#from imblearn.over_sampling import RandomOverSampler

import warnings
warnings.filterwarnings('ignore')

import pickle
from pickle import dump, load

np.random.seed(0)

from pycaret.classification import *
#from pycaret.clustering import *
#from pycaret.regression import *

pd.set_option('display.max_columns',100)
#pd.set_option('display.max_rows',100)
pd.set_option('display.width', 1000)

### Data Exploration and Analysis

In [2]:
df = pd.read_csv("wk4.csv")

In [3]:
df

,Retailer,Salerank,X2013USSales,X2013WorldSales,ProfitMargin,NumStores,Industry,Reward,ProgramName,RewardType,RewardStructure,RewardSize,ExpirationMonth,IndustryType
0,A&P,74,5.831,5.831,48.85,0.277,"Discount, Variety Stores",0,No rewards program,-,-,-,-,Discount
1,Albertsons,21,19.452,19.452,69.02,1.024,Grocery Stores,0,No rewards program,-,-,-,-,Grocery
2,Aldi,38,10.898,10.650,69.41,1.328,Grocery Stores,0,No rewards program,-,-,-,-,Grocery
3,Alimentation Couche Tard (Circle K),82,4.755,8.551,68.03,3.826,Grocery Stores,0,No rewards program,-,-,-,-,Grocery
4,Apple Stores,15,26.648,30.736,11.07,0.254,ElectronicEquipment,0,No rewards program,-,-,-,-,Specialty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Toys ""R"" Us",57,7.525,13.307,79.33,0.868,"Specialty Retail, Other",1,"Rewards""R"" Us",store credit,Spend $125 earn $5 Reward Dollars for future p...,4,12,Specialty
96,Tractor Supply,77,5.165,5.165,2.65,1.276,"Specialty Retail, Other",1,Neighbor's Club,store credit,Spend $150 X3 times earn seasonal (quarterly) ...,1.11,3,Specialty
97,True Value,27,16.330,16.330,38.69,4.494,Home Improvement Stores,1,TrueValue Rewards,store credit,"Spend $250 earn 2,500 points = $5 reward certi...",2,24,Specialty
98,Walgreen,6,68.068,70.096,23.47,7.998,Drug Stores,1,Balance Rewards,store credit,500 points on every prescription filled in the...,10,6,Specialty


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Retailer         100 non-null    object 
 1   Salerank         100 non-null    int64  
 2   X2013USSales     100 non-null    float64
 3   X2013WorldSales  100 non-null    float64
 4   ProfitMargin     100 non-null    float64
 5   NumStores        100 non-null    float64
 6   Industry         100 non-null    object 
 7   Reward           100 non-null    int64  
 8   ProgramName      100 non-null    object 
 9   RewardType       96 non-null     object 
 10  RewardStructure  100 non-null    object 
 11  RewardSize       100 non-null    object 
 12  ExpirationMonth  100 non-null    object 
 13  IndustryType     100 non-null    object 
dtypes: float64(4), int64(2), object(8)
memory usage: 11.1+ KB


In [5]:
df.describe(include='all')

,Retailer,Salerank,X2013USSales,X2013WorldSales,ProfitMargin,NumStores,Industry,Reward,ProgramName,RewardType,RewardStructure,RewardSize,ExpirationMonth,IndustryType
count,100,100.000000,100.000000,100.000000,100.00000,100.000000,100,100.00,100,96,100,100,100,100
unique,100,NaN,NaN,NaN,NaN,NaN,17,NaN,58,11,57,27,11,5
top,Dollar General,NaN,NaN,NaN,NaN,NaN,"Discount, Variety Stores",NaN,No rewards program,-,-,-,-,Specialty
freq,1,NaN,NaN,NaN,NaN,NaN,18,NaN,40,37,41,44,44,43
mean,NaN,50.500000,18.373500,24.131540,45.27300,2.698760,NaN,0.55,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,29.011492,36.476003,50.845864,29.23139,3.997641,NaN,0.50,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,1.000000,3.600000,3.600000,1.02000,0.000000,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,25.750000,5.206750,6.108250,19.44500,0.338500,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,50.500000,8.348500,9.629000,42.02000,1.331500,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,75.250000,16.841000,22.131500,69.57250,3.519750,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.shape

(100, 14)

In [7]:
df.columns

Index(['Retailer', 'Salerank', 'X2013USSales', 'X2013WorldSales', 'ProfitMargin', 'NumStores', 'Industry', 'Reward', 'ProgramName', 'RewardType', 'RewardStructure', 'RewardSize', 'ExpirationMonth', 'IndustryType'], dtype='object')

### Drop unwanted features

In [8]:
df.drop(['Retailer', 'Industry','ProgramName', 'RewardType', 'RewardStructure', 'RewardSize', 'ExpirationMonth'],axis=1,inplace=True)

In [9]:
df

,Salerank,X2013USSales,X2013WorldSales,ProfitMargin,NumStores,Reward,IndustryType
0,74,5.831,5.831,48.85,0.277,0,Discount
1,21,19.452,19.452,69.02,1.024,0,Grocery
2,38,10.898,10.650,69.41,1.328,0,Grocery
3,82,4.755,8.551,68.03,3.826,0,Grocery
4,15,26.648,30.736,11.07,0.254,0,Specialty
...,...,...,...,...,...,...,...
95,57,7.525,13.307,79.33,0.868,1,Specialty
96,77,5.165,5.165,2.65,1.276,1,Specialty
97,27,16.330,16.330,38.69,4.494,1,Specialty
98,6,68.068,70.096,23.47,7.998,1,Specialty


In [10]:
df2 = pd.get_dummies(df, drop_first=True)

In [11]:
df2

,Salerank,X2013USSales,X2013WorldSales,ProfitMargin,NumStores,Reward,IndustryType_Discount,IndustryType_Grocery,IndustryType_Restaurants,IndustryType_Specialty
0,74,5.831,5.831,48.85,0.277,0,1,0,0,0
1,21,19.452,19.452,69.02,1.024,0,0,1,0,0
2,38,10.898,10.650,69.41,1.328,0,0,1,0,0
3,82,4.755,8.551,68.03,3.826,0,0,1,0,0
4,15,26.648,30.736,11.07,0.254,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
95,57,7.525,13.307,79.33,0.868,1,0,0,0,1
96,77,5.165,5.165,2.65,1.276,1,0,0,0,1
97,27,16.330,16.330,38.69,4.494,1,0,0,0,1
98,6,68.068,70.096,23.47,7.998,1,0,0,0,1


In [12]:
df2.columns

Index(['Salerank', 'X2013USSales', 'X2013WorldSales', 'ProfitMargin', 'NumStores', 'Reward', 'IndustryType_Discount', 'IndustryType_Grocery', 'IndustryType_Restaurants', 'IndustryType_Specialty'], dtype='object')

In [13]:
df3 = df2[['Salerank', 'X2013USSales', 'X2013WorldSales', 'ProfitMargin', 'NumStores',
        'IndustryType_Discount', 'IndustryType_Grocery', 'IndustryType_Restaurants', 'IndustryType_Specialty',
        'Reward']]

In [14]:
df3

,Salerank,X2013USSales,X2013WorldSales,ProfitMargin,NumStores,IndustryType_Discount,IndustryType_Grocery,IndustryType_Restaurants,IndustryType_Specialty,Reward
0,74,5.831,5.831,48.85,0.277,1,0,0,0,0
1,21,19.452,19.452,69.02,1.024,0,1,0,0,0
2,38,10.898,10.650,69.41,1.328,0,1,0,0,0
3,82,4.755,8.551,68.03,3.826,0,1,0,0,0
4,15,26.648,30.736,11.07,0.254,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
95,57,7.525,13.307,79.33,0.868,0,0,0,1,1
96,77,5.165,5.165,2.65,1.276,0,0,0,1,1
97,27,16.330,16.330,38.69,4.494,0,0,0,1,1
98,6,68.068,70.096,23.47,7.998,0,0,0,1,1


### Treat Missing Values

In [15]:
df3.isnull().sum()

Salerank                    0
X2013USSales                0
X2013WorldSales             0
ProfitMargin                0
NumStores                   0
IndustryType_Discount       0
IndustryType_Grocery        0
IndustryType_Restaurants    0
IndustryType_Specialty      0
Reward                      0
dtype: int64

### Create and save processed dataset

In [16]:
#df.to_csv("week4train.csv",index=False)

In [17]:
df3.shape

(100, 10)

### Train Test Split

In [18]:
X = df3.iloc[:,0:9]
y = df3.iloc[:,9]

In [19]:
X.values, y.values

(array([[7.40000e+01, 5.83100e+00, 5.83100e+00, 4.88500e+01, 2.77000e-01,
         1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00],
        [2.10000e+01, 1.94520e+01, 1.94520e+01, 6.90200e+01, 1.02400e+00,
         0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00],
        [3.80000e+01, 1.08980e+01, 1.06500e+01, 6.94100e+01, 1.32800e+00,
         0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00],
        [8.20000e+01, 4.75500e+00, 8.55100e+00, 6.80300e+01, 3.82600e+00,
         0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00],
        [1.50000e+01, 2.66480e+01, 3.07360e+01, 1.10700e+01, 2.54000e-01,
         0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00],
        [4.80000e+01, 8.64000e+00, 1.63010e+01, 6.04100e+01, 5.30000e-01,
         0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00],
        [5.10000e+01, 8.34700e+00, 8.09600e+00, 4.69500e+01, 2.17900e+00,
         0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00],
        [7.10000e+01, 6.08200e+00, 6.08200e+00, 

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.4, random_state=0)

In [21]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((60, 9), (40, 9), (60,), (40,))

### Feature Scaling

In [22]:
X_train

array([[2.10000e+01, 1.94520e+01, 1.94520e+01, 6.90200e+01, 1.02400e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00],
       [6.90000e+01, 6.21200e+00, 6.21200e+00, 1.53400e+01, 6.44000e-01,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00],
       [3.20000e+01, 1.24910e+01, 1.23660e+01, 3.28800e+01, 3.47000e-01,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00],
       [4.50000e+01, 9.08300e+00, 8.89200e+00, 1.67000e+01, 5.79100e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00],
       [1.50000e+01, 2.66480e+01, 3.07360e+01, 1.10700e+01, 2.54000e-01,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00],
       [2.30000e+01, 1.88170e+01, 2.91100e+01, 8.42300e+01, 1.51400e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00],
       [2.50000e+01, 1.75040e+01, 1.75040e+01, 5.41200e+01, 1.11320e+01,
        1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00],
       [6.20000e+01, 6.99900e+00, 6.99900e+00, 4.59300e+01, 8.

In [23]:
minmax = MinMaxScaler()

In [24]:
X_train_scaled = minmax.fit_transform(X_train)

In [25]:
X_test_scaled = minmax.transform(X_test)

In [26]:
X_train_scaled

array([[2.04081633e-01, 4.77990655e-02, 3.33223450e-02, 6.92605419e-01,
        3.84326678e-02, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [6.93877551e-01, 7.75733015e-03, 5.16381501e-03, 1.45854553e-01,
        2.41705450e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00],
       [3.16326531e-01, 2.67469114e-02, 1.82520018e-02, 3.24506009e-01,
        1.30235700e-02, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [4.48979592e-01, 1.64400962e-02, 1.08635779e-02, 1.59706661e-01,
        2.17347245e-01, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00],
       [1.42857143e-01, 6.95619301e-02, 5.73208988e-02, 1.02363007e-01,
        9.53310314e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00],
       [2.24489796e-01, 4.58786348e-02, 5.38627591e-02, 8.47524954e-01,
        5.68232998e-02, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.0

In [27]:
X_test_scaled

array([[ 7.24489796e-01,  4.99311972e-03,  4.96177118e-03,
         2.76023630e-01,  4.68022819e-02,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 8.06122449e-01,  3.75618091e-03,  2.35008879e-03,
         1.86697902e-01,  8.48220988e-03,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.77551020e-01,  2.19292011e-02,  1.46024522e-02,
         6.96577714e-01,  4.98423660e-02,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.85714286e-01,  2.81804298e-02,  1.95259414e-02,
         5.57139947e-01,  7.54391233e-03,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 4.28571429e-01,  1.72445600e-02,  1.12782994e-02,
         6.45345284e-01,  9.93844768e-02,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 4.08163265e-02,  2.00523204e-01,  1.59567839e-01,
         5.84436749e-01,  7.37501877e-02,  0.000000

### Neural Network

In [28]:
nn = MLPClassifier()

In [29]:
nn.fit(X_train_scaled, y_train)

MLPClassifier()

In [30]:
y_pred = nn.predict(X_test_scaled)

In [31]:
y_pred

array([1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0], dtype=int64)

In [32]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Salerank                  100 non-null    int64  
 1   X2013USSales              100 non-null    float64
 2   X2013WorldSales           100 non-null    float64
 3   ProfitMargin              100 non-null    float64
 4   NumStores                 100 non-null    float64
 5   IndustryType_Discount     100 non-null    uint8  
 6   IndustryType_Grocery      100 non-null    uint8  
 7   IndustryType_Restaurants  100 non-null    uint8  
 8   IndustryType_Specialty    100 non-null    uint8  
 9   Reward                    100 non-null    int64  
dtypes: float64(4), int64(2), uint8(4)
memory usage: 5.2 KB


In [33]:
df3['Reward'].value_counts()

1    55
0    45
Name: Reward, dtype: int64

### Model Training

### Using PyCaret

In [34]:
exp = setup(data = df3, target = 'Reward', session_id=0, normalize=True, normalize_method='minmax', train_size=0.6) 

Setup Succesfully Completed!


,Description,Value
0,session_id,0
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(100, 10)"
4,Missing Values,False
5,Numeric Features,9
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [35]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,K Neighbors Classifier,0.5500,0.5931,0.5500,0.6000,0.5343,0.1182,0.1397,0.0069
1,Naive Bayes,0.5500,0.4889,0.6000,0.5933,0.5700,0.0900,0.1114,0.0053
2,Quadratic Discriminant Analysis,0.5500,0.5389,0.2167,0.5000,0.2800,0.1697,0.2080,0.0016
3,Extreme Gradient Boosting,0.5500,0.5403,0.5833,0.5733,0.5595,0.1133,0.0896,0.0738
4,Extra Trees Classifier,0.5167,0.4604,0.6000,0.5583,0.5586,0.0048,-0.0016,0.1623
5,Random Forest Classifier,0.4833,0.4368,0.5167,0.4833,0.4869,-0.0333,-0.0488,0.0301
6,Ada Boost Classifier,0.4833,0.5097,0.4917,0.4917,0.4693,-0.0047,-0.0256,0.0896
7,SVM - Linear Kernel,0.4667,0.0000,0.5833,0.4500,0.4638,-0.0937,-0.0947,0.0061
8,CatBoost Classifier,0.4667,0.5208,0.6000,0.4950,0.5282,-0.0712,-0.0815,1.1628
9,Logistic Regression,0.4333,0.3917,0.6667,0.4833,0.5476,-0.1667,-0.1748,0.0062


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='uniform')

In [36]:
dt = create_model('dt')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.3333,0.3750,0.2500,0.5000,0.3333,-0.2000,-0.2500
1,0.1667,0.1250,0.2500,0.3333,0.2857,-0.6667,-0.7071
2,0.8333,0.8750,0.7500,1.0000,0.8571,0.6667,0.7071
3,0.6667,0.6667,0.6667,0.6667,0.6667,0.3333,0.3333
4,0.1667,0.1667,0.0000,0.0000,0.0000,-0.6667,-0.7071
5,0.3333,0.3333,0.3333,0.3333,0.3333,-0.3333,-0.3333
6,0.5000,0.5000,0.6667,0.5000,0.5714,0.0000,0.0000
7,0.1667,0.1667,0.0000,0.0000,0.0000,-0.6667,-0.7071
8,0.5000,0.5000,0.6667,0.5000,0.5714,0.0000,0.0000
9,0.3333,0.3333,0.3333,0.3333,0.3333,-0.3333,-0.3333


In [37]:
print(dt)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')


In [38]:
evaluate_model(dt)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [39]:
dtbag = create_model('dt', ensemble=True, method='Bagging')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.1667,0.0000,0.0000,0.0000,0.0000,-0.3636,-0.6325
1,0.3333,0.3750,0.5000,0.5000,0.5000,-0.5000,-0.5000
2,0.6667,0.6250,0.7500,0.7500,0.7500,0.2500,0.2500
3,0.5000,0.5556,0.6667,0.5000,0.5714,0.0000,0.0000
4,0.6667,0.6667,1.0000,0.6000,0.7500,0.3333,0.4472
5,0.5000,0.6111,0.3333,0.5000,0.4000,0.0000,0.0000
6,0.6667,0.7222,0.6667,0.6667,0.6667,0.3333,0.3333
7,0.0000,0.0000,0.0000,0.0000,0.0000,-1.0000,-1.0000
8,0.6667,0.5556,0.6667,0.6667,0.6667,0.3333,0.3333
9,0.0000,0.0000,0.0000,0.0000,0.0000,-1.0000,-1.0000


In [40]:
print(dtbag)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=0,
   

In [41]:
evaluate_model(dtbag)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [42]:
dtboost = create_model('dt', ensemble=True, method='Boosting')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.3333,0.3750,0.2500,0.5000,0.3333,-0.2000,-0.2500
1,0.1667,0.1250,0.2500,0.3333,0.2857,-0.6667,-0.7071
2,0.6667,0.7500,0.5000,1.0000,0.6667,0.4000,0.5000
3,0.6667,0.6667,0.6667,0.6667,0.6667,0.3333,0.3333
4,0.1667,0.1667,0.0000,0.0000,0.0000,-0.6667,-0.7071
5,0.5000,0.5000,0.3333,0.5000,0.4000,0.0000,0.0000
6,0.3333,0.3333,0.6667,0.4000,0.5000,-0.3333,-0.4472
7,0.1667,0.1667,0.0000,0.0000,0.0000,-0.6667,-0.7071
8,0.5000,0.5000,0.6667,0.5000,0.5714,0.0000,0.0000
9,0.3333,0.3333,0.3333,0.3333,0.3333,-0.3333,-0.3333


In [43]:
print(dtboost)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                       

In [44]:
evaluate_model(dtboost)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…